Goal: Allow the user to create a directory of small images (of the same size?) and use them as a data set to mask an image.  This example code will try to find the closest match based on RGB similarties between image region and sample images.

In [1]:
from PIL import Image
from os import listdir
from IPython.core.display import display, HTML
# NOTE: update to work with different image source
#emojiCharSet_24x24_35.png

In [2]:
# Read average RGB values for this image
def imageRGBs( image ):
    #image = 'files/imageset/csharp.png'
    im = Image.open(image)
    width, height = im.size
    pix = im.load()
    r = 0
    g = 0
    b = 0
    count = width*height
    for w in range(width):
        for h in range(height):
            r += pix[w,h][0]
            g += pix[w,h][1]
            b += pix[w,h][2]
    print(image,(int(r/count), int(g/count),int(b/count)))
    return (image,(int(r/count), int(g/count),int(b/count)))

In [3]:
# TODO: Read all images in a folder and create a dictionary of image:(rgb) data.
filepath = 'imageset'
print(len(listdir(filepath)))
for f in listdir(filepath):
    print(filepath+'/'+f)
    imageRGBs(filepath+'/'+f)

18
imageset/apple.png
imageset/apple.png (209, 188, 168)
imageset/apple2.png
imageset/apple2.png (229, 232, 233)
imageset/cpp.png
imageset/cpp.png (144, 190, 222)
imageset/csharp.png
imageset/csharp.png (194, 157, 196)
imageset/css.png
imageset/css.png (172, 205, 225)
imageset/droid.png
imageset/droid.png (215, 231, 169)
imageset/github.png
imageset/github.png (158, 157, 157)
imageset/html.png
imageset/html.png (235, 192, 179)
imageset/java.png
imageset/java.png (233, 229, 239)
imageset/javascript.png
imageset/javascript.png (200, 222, 171)
imageset/jQuery.png
imageset/jQuery.png (230, 234, 236)
imageset/jupyter.png
imageset/jupyter.png (240, 224, 214)
imageset/numpy.png
imageset/numpy.png (187, 198, 200)
imageset/php.png
imageset/php.png (184, 193, 207)
imageset/php2.png
imageset/php2.png (232, 236, 243)
imageset/python.png
imageset/python.png (60, 65, 46)
imageset/python2.png
imageset/python2.png (201, 205, 182)
imageset/vstudio.png
imageset/vstudio.png (182, 147, 190)


In [12]:
def makeCharImageSet( directory, height ):
    '''inputs: directory of images, height in pixels of output imageset
    outputs: filename of output imageset'''
    width = height*len(listdir(directory))
    # Create shell of resulting image
    resultImg = Image.new('RGB', (width,height))
    count = 0
    for image in listdir(directory):
        path = directory + '/' + image
        im = Image.open( path )
        resultImg.paste(im, (count*height,0))
        count += 1
    resultImg.save('charImageSet.jpg','JPEG')
    display(HTML('<img src="charImageSet.jpg">'))
    return 'charImageSet.jpg'

In [7]:
makeCharImageSet('imageset', 48)

In [15]:
def get_RGB_dictionary(imageset, size):
    '''inputs: image containing uniform sized icons, size in pixels of each icon
    outputs: a list of icon index and average RGB values'''
    results = []  # store image index and average RGB tuples (1, (0,0,0))

    im = Image.open(imageset)
    width, height = im.size
    pix = im.load()
    index = 0
    for box in range(0,width,size):   # 0, 48, 96... moving x starting place
        r = 0
        g = 0
        b = 0
        for w in range(box, box+size):
            for h in range(height):
                r += pix[w,h][0]
                g += pix[w,h][1]
                b += pix[w,h][2]
        avgR = r/(size**2)
        avgG = g/(size**2)
        avgB = b/(size**2)
        results.append( (index, (avgR,avgG,avgB)) )
        index += 1
    return results

In [33]:
# find an image to convert to an alternate image set.  open this image and scale to a certain % size
def scaleInputImage( image_path, width_multiple, height_multiple, scale):
    im = Image.open(image_path)
    print(im.width, im.height)
    print(im.width-im.width%width_multiple,im.height-im.height%height_multiple)
    imresize = im.resize(((im.width-im.width%width_multiple)*scale, (im.height-im.height%height_multiple)*scale), Image.ANTIALIAS)

    newname = image_path.split('.')[0] + '_%dx%d.png' %(width_multiple,height_multiple)
    imresize.save(newname)
    return newname

In [30]:
def findClosestRGB( testTuple, replaceList ):
    # testTuple in form (avgR, avgG, argB)
    # replaceList in form [ (index, (avgR, avgG, avgB)), ... ]
    minDiff = 3*255
    minIndex = 0
    for icon in replaceList:
        idx = icon[0]
        iR = icon[1][0]
        iG = icon[1][1]
        iB = icon[1][2]
        diff = abs(testTuple[0]-iR) + abs(testTuple[1]-iG) + abs(testTuple[2]-iB)
        if diff < minDiff:
            minIndex = idx
            minDiff = diff
    return minIndex

In [34]:
# input arguments:
imageDirectory = 'imageset'
size = 48
testImage = 'test.jpg'

# STEP 1: Create a single image set out of a directory of sample images.
replaceImage = makeCharImageSet(imageDirectory, size)
print('(1) replacement imageset created:', replaceImage)

# STEP 2: Scan through the replaceImage file and create a list of
#         replaceImage avgerage RGB values
replaceRGBlist = get_RGB_dictionary(replaceImage, size)
print('(2) list of replacment RGB values created: %d items' %(len(replaceRGBlist)))
#print(replaceRGBlist,'\n')
    
# STEP 3: Scale test image so that it has %size dimensions
# TODO: update algorithm below so that this isn't required
testImage = scaleInputImage(testImage, size, size, 8)
print('(3) test image scaled. ready to be processed:', testImage)

# STEP 4: 
# scan through the scaled test image in chuncks of 48x48 (or custom) and find the average RGB for each region
# next, compare that average RGB against the test set to find the closest match.
# finally, replace this image with the test set image.
 
im = Image.open(testImage)
pix = im.load()

replaceImage = Image.open(replaceImage)
#charpix = charimg.load()

for boxX in range(0,im.width,size):
    for boxY in range(0,im.height,size):
        # loop over each box
        r = 0
        g = 0
        b = 0
        for x in range(boxX,boxX+size):
            for y in range(boxY, boxY+size):
                r += pix[x,y][0]
                g += pix[x,y][1]
                b += pix[x,y][2]
        # average the r,g,b pixel values
        rAvg = r/(size*size)
        gAvg = g/(size*size)
        bAvg = b/(size*size)
        avgRGB = (rAvg, gAvg, bAvg)
        # next, compare this tuple of averaged RGB values
        # against the averaged RGB tuples in the replacement image set
        # the method will return the location in the replacement image
        # set with the minimum difference
        
        index = findClosestRGB( avgRGB, replaceRGBlist )
        #print(avgRGB,'index',index)
        #top,bot = topBot(pix, i, j, 48, 48)
        
        #index = picChar(_charDict, (top,bot))
        
        chop = replaceImage.crop((index*size,0,index*size+size,size))
        im.paste(chop, (boxX,boxY,boxX+size,boxY+size))
        
im.save('result.png')
print('Done.')






(1) replacement imageset created: charImageSet.jpg
(2) list of replacment RGB values created: 18 items
1180 842
1152 816
(3) test image scaled. ready to be processed: test_48x48.png
Done.


In [24]:
def topBot( pix, xo, yo, width, height ):
    top = 0
    bot = 0
    for x in range(xo, xo+width):  # was +9
        for y in range(yo, yo+height//2):  # was +8
            # find average top half
            avg = (pix[x,y][0] + pix[x,y][1] + pix[x,y][2]) / 3
            top += avg 
        for y in range(yo+height//2,yo+height):  # was +8 +16
            # find average bottom half
            avg = (pix[x,y][0] + pix[x,y][1] + pix[x,y][2]) / 3
            bot += avg
    top = top // (width*height//2)
    bot = bot // (width*height//2)
    return top,bot

In [25]:
def picChar( cDict, topBotTup ):
    # return the index of the dictionary with the closest match
    _minDiff = 512
    _minIndex = 0
    top = topBotTup[0]
    bot = topBotTup[1]
    for key in cDict:
        t = cDict[key][0]
        b = cDict[key][1]
        diff = (max(t,top)-min(t,top)) + (max(b,bot)-min(b,bot))
        if diff < _minDiff:
            _minDiff = diff
            _minIndex = key
    return _minIndex

In [47]:
#charset = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
#charset += '.,/;\'<>?:"[]{}\\|`1234567890-=~!@#$%^&*()_+ '
#FilePath = 'Files\\CharacterData.png'

# alternate attempt
#charset = 'robctpusROBCTPUS '
#FilePath = 'Files\\CharacterDataRob.png'

# mario data
#FilePath = 'Files\\marioCharSet.png'
#_w = 16
#_h = 32
#_count = 320

# emoji data
#FilePath = 'Files\\emojiCharSet_20x20_93.png'
#_w = 20
#_h = 20
#_count = 93

FilePath = 'charImageSet.jpg'
_w = 48
_h = 48
_count = 16

# I need to create a dictionary of character pixel information in the form:
# (index: (top_intensity,bottom_intensity))  i.e. (0: (10,20))
# Note: FilePath needs to be the path to an image file containing fixed-width
# font characters on a single line.
# Note 2: This algorithm is hard-coded for 12-point Consolas font with a
#         character width of 9 pixels and a character height of 16 pixels
im = Image.open(FilePath)
pix = im.load()
_charDict = {}
for i in range(_count):
    top,bot = topBot( pix, i*_w, 0, _w, _h)
    _charDict[i] = (top,bot)

# Next, determine what the lowest intensity value is so we can normalize to this 
# value (otherwise we get images full of '@' characters because average images
# are dark!)
lowest = 255
for key in _charDict:
    if( _charDict[key][0] < lowest ):
        lowest = _charDict[key][0]
    if( _charDict[key][1] < lowest ):
        lowest = _charDict[key][1]
norm = 255/(255-lowest)
print(lowest, norm)

# normalize dictionary
for key in _charDict:
    normTop = int((_charDict[key][0] - lowest)*norm)
    normBot = int((_charDict[key][1] - lowest)*norm)
    _charDict[key] = ( normTop, normBot )
print(_charDict)

50.0 1.2439024390243902
{0: (191, 146), 1: (233, 215), 2: (170, 161), 3: (170, 153), 4: (180, 190), 5: (191, 190), 6: (121, 143), 7: (177, 194), 8: (228, 218), 9: (185, 179), 10: (211, 216), 11: (169, 185), 12: (179, 179), 13: (0, 22), 14: (166, 194), 15: (149, 150)}


In [57]:
imageSet = 'charImageSet.jpg'
trainIM = Image.open(imageSet)
boxSize = trainIM.height
numImages = trainIM.width//boxSize
print(boxSize,numImages)

pixels = trainIM.load()

_colorPixDict = {}

for img in range(numImages):
    r = 0
    g = 0
    b = 0
    for x in range(img*boxSize, (img+1)*boxSize):
        for y in range(0, boxSize):
            r += pixels[x,y][0]
            g += pixels[x,y][1]
            b += pixels[x,y][2]
    tot = boxSize**2
    print(img,(int(r/tot), int(g/tot),int(b/tot)))
    _colorPixDict[img] = (int(r/tot), int(g/tot),int(b/tot))
    
_colorPixDict

48 16
0 (206, 187, 165)
1 (228, 231, 232)
2 (144, 189, 218)
3 (191, 156, 193)
4 (171, 204, 222)
5 (214, 230, 167)
6 (156, 156, 156)
7 (231, 191, 177)
8 (230, 227, 233)
9 (199, 221, 169)
10 (235, 221, 210)
11 (184, 196, 197)
12 (184, 192, 206)
13 (62, 66, 50)
14 (200, 204, 180)
15 (178, 146, 187)


{0: (206, 187, 165),
 1: (228, 231, 232),
 2: (144, 189, 218),
 3: (191, 156, 193),
 4: (171, 204, 222),
 5: (214, 230, 167),
 6: (156, 156, 156),
 7: (231, 191, 177),
 8: (230, 227, 233),
 9: (199, 221, 169),
 10: (235, 221, 210),
 11: (184, 196, 197),
 12: (184, 192, 206),
 13: (62, 66, 50),
 14: (200, 204, 180),
 15: (178, 146, 187)}

In [49]:
## ORIGINAL ALGORITHM

testfilename = 'test.jpg'
scale = 4
im = Image.open(testfilename)
print(im.width, im.height)
print(im.width-im.width%_w,im.height-im.height%_h)
imresize = im.resize(((im.width-im.width%_w)*scale, (im.height-im.height%_h)*scale), Image.ANTIALIAS)

newname = testfilename.split('.')[0] + 'r.png'
imresize.save(newname)

im = Image.open(newname)
pix = im.load()

charimg = Image.open(FilePath)
charpix = charimg.load()

for i in range(0,im.width,_w):
    for j in range(0,im.height,_h):
        top,bot = topBot(pix, i, j, _w, _h)
        index = picChar(_charDict, (top,bot))
        chop = charimg.crop((index*_w,0,index*_w+_w,_h))
        im.paste(chop, (i,j,i+_w,j+_h))
im.save(testfilename.split('.')[0] + '_emoji.png')

1000 618
960 576
